**Data Description**

You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

    toxic
    severe_toxic
    obscene
    threat
    insult
    identity_hate

You must create a model which predicts a probability of each type of toxicity for each comment.

**File descriptions**

    train.csv - the training set, contains comments with their binary labels
    test.csv - the test set, you must predict the toxicity probabilities for these comments. To deter hand labeling, the test set contains some comments which are not included in scoring.
    sample_submission.csv - a sample submission file in the correct format
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
submission = pd.read_csv('../input/sample_submission.csv')

In [ ]:
print('DataFrame Train:')
print(df_train.isnull().any())
print('\n')
print('DataFrame Test:')
print(df_test.isnull().any())

In [ ]:
df_test = df_test.fillna('unknown')

In [ ]:
col = np.array(df_train.columns)
col = col[2:]
print(col)

In [ ]:
print('Dataframe Train:')
for c in col:
    print("The dataframe has '{1}' of comments '{0}' of the total '{2}'.".format(c, 
                                                                                 df_train[c].sum(), 
                                                                                 len(df_train)))

In [ ]:
comment_text_all = pd.concat([df_train['comment_text'], df_test['comment_text']],axis=0)

In [ ]:
nrow_train = df_train.shape[0]
print(nrow_train)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', strip_accents='unicode')

In [ ]:
train_test_comment_text = vectorizer.fit_transform(comment_text_all)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [ ]:
y = df_train[col]
y.head(5)

In [ ]:
predict = np.zeros((df_test.shape[0], len(col)))
predict.shape

In [ ]:
loss = []
accuracy = []
for num, index in enumerate(col):
    x_train, x_test, y_train, y_test = train_test_split(train_test_comment_text[:nrow_train], y[index])
    print("Fit: {}".format(index))
    model = LogisticRegression(C=5)
    model.fit(x_train, y_train)
    predict[:,num] = model.predict_proba(train_test_comment_text[nrow_train:])[:,1]    
    predict_01 = model.predict_proba(x_test)[:,1]    
    logloss = log_loss(y_test, predict_01)
    print('log loss:', logloss)
    predict_02 = model.predict(x_test)
    acc = np.mean(predict_02 == y_test)
    print('accuracy:', acc)
    loss.append(logloss)
    accuracy.append(acc)
    print('\n')
print('mean column-wise log loss:', np.mean(loss))
print('mean column-wise accuracy:', np.mean(accuracy))

In [ ]:
submid = pd.DataFrame({'id': submission["id"]})
submission = pd.concat([submid, pd.DataFrame(predict, columns = col)], axis=1)
submission.to_csv('submission.csv', index=False)